In [49]:
import urllib.request
import requests
import json
import pandas as pd

In [3]:
# download a file from public repository online
# to temporary file

file_url = "https://www.dropbox.com/s/toyehkkfduogbwk/factors_gen.txt?dl=1"
filename = "factors_gen.txt"
urllib.request.urlretrieve(file_url, filename)

('factors_gen.txt', <http.client.HTTPMessage at 0x7fe9fcf4d2d0>)

In [45]:
myfile = open(filename, "r")

all_factors = []
factor = {}
while myfile:
    line  = myfile.readline()
    if "{" in line:
        all_factors.append(factor)
        factor = {}
        
    if ":" in line:
        var = line.split(":")
        factor[var[0].strip().strip("\n")] = line[(line.find(":")+1):].strip().strip("\n").strip('"')
        
    if line == "":
        break
        
myfile.close()


In [52]:
len(all_factors)

2577

In [46]:
# preview
all_factors[2:10]

[{'Index': '0',
  'CppName': 'FI_PAGE_RANK',
  'Name': 'PR',
  'Wiki': 'https://wiki.yandex-team.ru/jandekspoisk/kachestvopoiska/factordev/web/factors/PageRank',
  'AntiSeoUpperBound': '1.0',
  'Tags': '[TG_DOC, TG_LINK_GRAPH, TG_STATIC, TG_L2, TG_UNUSED]',
  'Description': 'Page rank. Фактор ремапится.',
  'Authors': 'aavdonkin',
  'Responsibles': 'aavdonkin'},
 {'Index': '1',
  'CppName': 'FI_TEXT_RELEV',
  'Name': 'TR',
  'AntiSeoUpperBound': '0.95',
  'Group': 'LegacyTR',
  'Tags': '[TG_DOC, TG_DOC_TEXT, TG_DYNAMIC, TG_REARR_USE, TG_UNDOCUMENTED, TG_NN_OVER_FEATURES_USE]',
  'Description': 'Текстовая релевантность (maxfreq – частота самого частого слова, которая имеет смысл длины документа).',
  'Authors': '["gulin", "iseg", "leo", "maslov"]',
  'Responsibles': '["gulin", "leo", "maslov"]'},
 {'Index': '2',
  'CppName': 'FI_LINK_RELEV',
  'Name': 'LR',
  'AntiSeoUpperBound': '1.0',
  'Group': 'Dynamic',
  'Tags': '[TG_DOC, TG_DYNAMIC, TG_LINK_TEXT, TG_UNDOCUMENTED, TG_DEPRECATED]',

# API translation

Input: Any language

Output: Translated to specified language

Signup at Deepl.com

In [29]:
api_key_deepl = "add key here"

# from the docs
#curl -X POST 'https://api-free.deepl.com/v2/translate' \
#	-H 'Authorization: DeepL-Auth-Key asdfasdfasdfa' \
#	-d 'text=Hello%2C%20world!' \
#	-d 'target_lang=DE'

def translate( text, target_lang="EN"):
    res = requests.post(
        'https://api-free.deepl.com/v2/translate',
        headers= {"Authorization":"DeepL-Auth-Key "+api_key_deepl},
        json = {"text":[text], "target_lang":target_lang}
    )

    return json.loads(res.text)['translations'][0]['text']

In [30]:

# test one input
translate('Приоритет strict для TR - текстовый приоритет - есть все слова запроса где-то в документе (при этом они проходят контекстные ограничения запроса, например, оба слова д.б. в одном предложении).')

'Priority strict for TR is text priority - there are all query words somewhere in the document (and they pass contextual restrictions of the query, for example, both words d.b. in the same sentence).'

In [47]:
# loop over every description and convert to english

i = 0

for factor in all_factors:
    
    if ('Desc' not in factor) and ('Description' in factor):
        print(i, factor['Description'])
        
        factor['Desc'] = translate(factor['Description'])
    i = i+1

2 Page rank. Фактор ремапится.
3 Текстовая релевантность (maxfreq – частота самого частого слова, которая имеет смысл длины документа).
4 Линковая релевантность. Фактор ремапится.
5 Priority bonus, приоритет 7  - текстовый приоритет. Фактор бинарный, имеет значение 0 для всех однословных запросов, и значение 1 практически для всех двух и более словных, кроме очень маленького количества ответов, для которых нет ни одной ссылки, прошедшей кворум, и текст тоже не прошел кворум.
6 Приоритет strict для TR - текстовый приоритет - есть все слова запроса где-то в документе (при этом они проходят контекстные ограничения запроса, например, оба слова д.б. в одном предложении).
7 Приоритет phrase для TR - текстовый приоритет - есть все слова запроса подряд в документе.
8 (strict) есть все слова запроса в одном линке.
9 (phrase) есть все слова запроса подряд в одном линке.
10 Наличие точной фразы (текста запроса) в заголовке (если точнее, в первом предложении документа). Контекстные ограничения и с

85 В ссылках есть все слова запроса
86 В одной ссылке есть все слова запроса
87 Есть ссылка, прошедшая кворум
88 Какая доля ссылок “хорошая”
89 Насколько много “плохих” ссылок (плохой = dpr = 0)
90 Максимальный dpr ссылки
91 TfIdf обычный TF*IDF по ссылкам. Частота слова в ссылках умножается на обратную документную частоту и суммируется по всем словам, потом нормируется на длину документа.
92 Линковая релевантность от Гулина
93 Линковая релевантность от Гулина
94 Линковая релевантность от Гулина
95 Есть точная форма всех слов запроса в тексте/линках
96 Есть лемма всех слов запроса в тексте/линках
97 Документ прошел softand по ограничениям синтаксического колдунщика. Только для документов, имеющих текстовую релевантность. Для однословных запросов всегда 1.
98 Классификатор качества входящих ссылок 2 - сломан, см [407]
99 равен единице, если у сайта стоит украинский гео-атрибут(т.е 1 - украинский сайт)
100 Страница с блогохостинга
101 Страница с livejournal.com
103 Автоматичемкий классиф

212 Вес максимального совпадения форм в тексте и запросе
213 На странице есть про 'оплату SMS'.
214 Антиспамеры пессимизировали сайт - все динамические линковые факторы обнуляются. zerolnk.flt
215 Магазинность страницы
216 Порнографичность страницы
217 Remapped mascot feature TrafgraphOutAll_share_m
218 Remapped mascot feature TrafgraphOutAllSE_share_d
219 Remapped mascot feature TrafgraphOutAllSE_share_m
220 Remapped mascot feature NoExtClicksShare
221 Поисковый трафик - переходы с поисковых систем на сайт (2-ая формула)
222 Поисковый трафик - переходы с поисковых систем на сайт (2-ая формула)
223 Переходы на сайт из поисковиков по отдельным словам, по данным бара
224 Значение фактора BclmMixPlainK000001 для стрима AliceMusic
225 Наибольшая общая подстрока урла и запроса, нормированная длиной урла
226 Все совпадения только в URL'е, в тексте страницы совпадений нет
229 Три уровня совпадения географии пользователя и страницы
232 Три уровня совпадения региона ссылок и запроса
233 Географ

326 Отношение суммы idf встретившихся слов в предложении+тайтл ко всем словам.
327 То же самое что JokerLen, по точным формам
328 То же самое что JokerWeight, по точным формам
329 Remapped mascot feature More120SecVisitsNotSearchShare
330 Аналоги соответствующих текстовых факторов для линков.  BM25 от количества ссылок, в которых произошло совпадение.
331 Простой BM25 по точной форме в текстах ссылок
332 Наличие в ссылках  пар слов c учетом синонимов
333 Количество ссылок, прошедших порог
334 Простой BM25 по ссылкам  c учетом синонимов
336 Запрос про видео
337 Кликабельность владельца независимо от запроса, отдельно по регионам
338 Энтропия - распределение кликов. Регионализованный
339 Энтропия - распределение показов. Регионализованный
340 Энтропия - распределение отношения клики/показы. Регионализованный
341 equals 2 * NastyContent
342 equals 2 * NastyContent
343 always zero
344 Домен в зоне .com
345 Домен в зоне .ua
346 Домен не в зоне .ru
347 LR по ссылкам из Яндекс.Маркета
348 Сти

416 Число входящих ссылок без русских букв. Ремапится.
417 Максимальное число форм по всем словам запроса - max по всем словам запроса числа_форм_для_слова/64
418 Взвешенная по весам слов сумма числа форм -  сумма по всем словам запроса числа_форм_для_слова/64*вес_слова; remap вида x/(1 + x).
419 Невзвешенная сумма числа форм - сумма по всем словам запроса числа_форм_для_слова/64/число_слов_запроса
420 Максимальное число форм по всем словам запроса
421 Взвешенная по весам слов сумма числа форм
422 Невзвешенная сумма числа форм
423 Аналоги одноименных факторов, вес слова = 1
424 Аналоги одноименных факторов, вес слова = 1
425 Аналоги одноименных факторов, вес слова = 1
426 Аналоги одноименных факторов, вес слова = 1
427 Аналоги одноименных факторов, вес слова = 1
428 Аналоги одноименных факторов, вес слова = 1
429 Сегменты запроса - это части запроса, которые сами по себе являются частотными запросами.  Фактор показывает, насколько сегменты бьются в тексте.   значение 0 - все  слова вст

...

1824 AVG aggregation of IsMusic web factor using random log
1826 PERCENTALE_25 aggregation of QueryThEncyclopedic web factor using random log
1828 AVG aggregation of CommercialOwnerRank_Reg web factor using random log
1830 PERCENTALE_25 aggregation of YabarWordDepthNodesGradientMin web factor using random log
1832 AVG aggregation of PopularSEFRCBrowser web factor using random log
1834 AVG aggregation of URLClicksMaxGeoRegionFRCRatio web factor using random log
1836 PERCENTALE_90 aggregation of UBLongPeriodDirectHChildren90CntFromExtHost web factor using random log
1838 PERCENTALE_90 aggregation of UBLongPeriodDtUrlHChildrenCut600Reg web factor using random log
1840 AVG aggregation of IsPicture web factor using random log
1842 AVG aggregation of ErratumLogQueryProbability web factor using random log
1848 Предсказанная по запросу и стране, с помощью dssm-модели длина клика из данной страны.
1851 Предсказанное с помощью нейронной сети среднее значение News по запросу за год.
1855 Предсказ

2040 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Фактор: MinWindowSize по стриму Body. Суммарное взвешенное значений фактора умноженных на вес (\\frac{\\Sum W_i * (W_i * F_i)}{\\Sum W_i}) нормированное на суммарный вес.
2042 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Фактор: Norm256 по стриму Bocm11. Суммарное взвешенное значений фактора умноженных на вес (\\frac{\\Sum W_i * (W_i * F_i)}{\\Sum W_i}) по топу расширений.
2045 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Минимальный вес расширения.
2046 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Среднее арифметическое весов расширений.
2048 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Суммарный вес расширений.
2050 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfDtShowKnn. Фактор: мешочный OriginalRequestFraction по стриму FieldSetBagOfWords.
2052 Фактор для экспер

2223 RMSE aggregation of Long web factor using random log
2225 RMSE aggregation of IsOrg web factor using random log
2227 RMSE aggregation of GskUrlModel web factor using random log
2229 RMSE aggregation of DaterStatsAverageSourceSegment web factor using random log
2231 RMSE aggregation of VisitsFromWiki web factor using random log
2233 RMSE aggregation of XfDtShowBagOfWordsTitleCosineMaxMatch web factor using random log
2235 RMSE aggregation of UBLongPeriodDownloadsProb web factor using random log
2237 RMSE aggregation of MetaAvgIsNotCgi meta factor using random log
2239 RMSE aggregation of MetaRmsSynPercentBadWordPairs meta factor using random log
2241 RMSE aggregation of MetaPosTrigramsProb meta factor using random log
2243 PERCENTALE_90 aggregation of Bocm web factor using random log
2245 PERCENTALE_90 aggregation of SegmentWordPortionFromMainContent web factor using random log
2247 PERCENTALE_90 aggregation of IsMobileBeauty web factor using random log
2249 PERCENTALE_90 aggregati

2386 Owner aggregation of PercentVisibleContent web factor using random log, aggregation type is PERCENTALE_90
2388 Owner aggregation of MetaWeb1241Web1299ProductInvPos meta factor using random log, aggregation type is PERCENTALE_25
2390 Owner aggregation of LinkAnnFloatMultiplicityAttenV1Bm15K001 web factor using random log, aggregation type is LOGAVG
2392 Owner aggregation of UBLongPeriodLeavesCnt web factor using random log, aggregation type is RMSE
2394 Owner aggregation of NumLinksFromMP web factor using random log, aggregation type is LOGAVG
2396 Owner aggregation of DssmRandomLogQueryAvgDifferentInternalLinks web factor using random log, aggregation type is PERCENTALE_25
2399 Owner aggregation of IsOrg web factor using random log, aggregation type is RMSE
2401 Owner aggregation of QSegmentsBM25 web factor using random log, aggregation type is MAX
2403 Owner aggregation of SegmentAuxAlphasInText web factor using random log, aggregation type is RMSE
2405 Owner aggregation of Rando

2499 Фактор лингвистического бустинга. Тип расширений: XfOneSeKnn (ближайшие по dssm-модели, обучаемой предсказывать XfDtShow расширения). Агрегация по всем расширениям. Наибольшее взвешенное значение фактора. Нормировано на максимальный вес расширения. Взвешенное объединение стримов Url,Title,Body,Links,CorrectedCtr,LongClick,OneClick,BrowserPageRank,SplitDwellTime,SamplePeriodDayFrc,SimpleClick,YabarVisits,YabarTime. Алгоритм агрегации весов слов: Bm15FLog (Bm15 агрегация логарифмов встречаемости слов). Коэффициент нормализации 0.001.
2501 Фактор лингвистического бустинга. Тип расширений: XfOneSeKnn (ближайшие по dssm-модели, обучаемой предсказывать XfDtShow расширения). Агрегация по всем расширениям. Наибольшее взвешенное значение фактора. Нормировано на максимальный вес расширения. TODO Алгоритм: максимальный вес полностью совпавшей с запросом аннотации. Считается по стриму OneClick.
2503 Фактор лингвистического бустинга. Тип расширений: QueryToTextByXfOneSeKnn (QueryToText расшире

# Pivot data

In [78]:
# convert list into columns

distinct_tags = {}

for factor in all_factors:
    
    # convert text to array
    tags = factor.get('Tags',"[]").strip("]").strip("[").split(",")
    
    for tag in tags:
        tag = tag.strip()
        # aggregate each tag
        distinct_tags[tag] = distinct_tags.get(tag,0)+1
        
        # pivot into column
        factor[tag] = True

In [77]:
sorted(distinct_tags.items(), key=lambda x:x[1], reverse=True)

[('TG_DOC', 1155),
 ('TG_DYNAMIC', 1154),
 ('TG_L2', 1030),
 ('TG_USER', 1019),
 ('TG_DEPRECATED', 999),
 ('TG_NN_OVER_FEATURES_USE', 902),
 ('TG_USER_SEARCH', 900),
 ('TG_STATIC', 671),
 ('', 667),
 ('TG_UNDOCUMENTED', 574),
 ('TG_USERFEAT', 508),
 ('TG_LOCALIZED_COUNTRY', 408),
 ('TG_USER_SEARCH_ONLY', 408),
 ('TG_DOC_TEXT', 383),
 ('TG_ANNOTATION_NOFILTER', 345),
 ('TG_TEXT_MACHINE', 330),
 ('TG_OFTEN_ZERO', 304),
 ('TG_USERFEAT_90D', 259),
 ('TG_SAMOHOD_UNIMPLEMENTED', 253),
 ('TG_CALLISTO_UNIMPLEMENTED', 250),
 ('TG_QUERY_ONLY', 248),
 ('TG_UNUSED', 242),
 ('TG_HOST', 222),
 ('TG_REARR_USE', 216),
 ('TG_RANDOM_LOG', 199),
 ('TG_NEURAL', 196),
 ('TG_BINARY', 192),
 ('TG_BROWSER', 185),
 ('TG_URL_TEXT', 153),
 ('TG_UNIMPLEMENTED', 149),
 ('TG_LINK_TEXT', 146),
 ('TG_USER_EXT_DATA', 143),
 ('TG_OWNER', 141),
 ('TG_FORMULA_2245_DEP_2', 137),
 ('TG_USERFEAT_CLICK_MACHINE', 133),
 ('TG_LINGBOOST', 126),
 ('TG_FORMULA_2245_DEP_3', 120),
 ('TG_REMOVED', 115),
 ('TG_USERFEAT_SEARCH_DWELL_T

# Convert to DataFrame

In [81]:
df = pd.DataFrame.from_dict(all_factors)

# Not Implemented but Named Factors

In [94]:
df_non = df[['Name','Desc','TG_UNIMPLEMENTED','TG_DEPRECATED']]
df_non[(df_non['TG_UNIMPLEMENTED']==True) | (df_non['TG_DEPRECATED']==True)][1:10]

,Name,Desc,TG_UNIMPLEMENTED,TG_DEPRECATED
8,LRp1,(strict) have all query words in one link.,NaN,True
9,LRp2,(phrase) have all query words in a row in one ...,NaN,True
14,Shop,This is a store offer (determined by the chara...,NaN,True
24,Geo,Indicates a match between the user's region an...,NaN,True
25,SubqueryThMatch,Matching thematic spectra of the query and the...,NaN,True
34,LRHitNum100,The converted number of query words in all url...,NaN,True
35,LRHitNumGt16,The document LR>20 has the number of occurrenc...,NaN,True
36,PctLinks,For documents with high LR - normalized link r...,NaN,True
37,HasLR,Url high LR.,NaN,True
